In [1]:
import csv,re
import numpy as np

In [4]:
fname = 'CRE

In [5]:
with open(fname) as f:
    all_rows = [r for r in csv.DictReader(f)]

In [6]:
column_map = {}
for k in all_rows[0].keys():
    m = re.match('^(\d+)_Rating$',k)
    if m:
        column_map[all_rows[0][k].split(" - ")[0]] = m[1]

In [7]:
all_rows[-1]

{'StartDate': '2023-07-27 08:09:05',
 'EndDate': '2023-07-27 08:16:17',
 'Status': 'IP Address',
 'IPAddress': '63.155.15.220',
 'Progress': '100',
 'Duration (in seconds)': '431',
 'Finished': 'True',
 'RecordedDate': '2023-07-27 08:16:18',
 'ResponseId': 'R_xm6FDUF9BRSyqM9',
 'RecipientLastName': '',
 'RecipientFirstName': '',
 'RecipientEmail': '',
 'ExternalReference': '',
 'LocationLatitude': '44.0563',
 'LocationLongitude': '-122.9164',
 'DistributionChannel': 'anonymous',
 'UserLanguage': 'EN',
 'WorkerID ': 'A26RO8GGTQAXGG',
 'Age': '59',
 'Sex': 'Female',
 'Education ': '16',
 'Occupation': 'clerk',
 '2_Rating': '6\nVery Much\n',
 '2_Timer_First Click': '2.516',
 '2_Timer_Last Click': '3.261',
 '2_Timer_Page Submit': '3.262',
 '2_Timer_Click Count': '2',
 '3_Rating': '0\nNot at All\n',
 '3_Timer_First Click': '2.312',
 '3_Timer_Last Click': '3.481',
 '3_Timer_Page Submit': '3.482',
 '3_Timer_Click Count': '2',
 '4_Rating': '0\nNot at All\n',
 '4_Timer_First Click': '2.717',
 '

In [8]:
catch_answers = [0,6,0,6,6]

In [9]:
def catch_acc(rr):
    try:
        return sum([float(rr[f'Catch{i+1}']==catch_answers[i]) for i in range(5)]) / 5.
    except:
        return 0.0

In [16]:
columns = ['WorkerID','Word','CatchAcc','Corr'] + list(column_map.keys())
rows = []
for r in all_rows[2:]:
    rr = {'WorkerID':r['WorkerID '],
         'Word':r['word']}
    for k in columns[4:]:
        val = r[f'{column_map[k]}_Rating'].strip().split()
        if len(val)>0:
            val = val[0]
            if val=='Not':
                val = 0
            val = int(val)
            rr[k] = val
    rr['CatchAcc'] = catch_acc(rr)
    rows.append(rr)

In [30]:
words = list(set([x['Word'] for x in rows]))

avg_vector = {}

for w in words:
    avg_vector[w] = []
    for c in columns[4:]:
        cc = []
        for r in filter(lambda x: x['Word']==w,rows):
            cc.append(r[c])
        avg_vector[w].append(sum(cc)/len(cc))

In [44]:
for r in rows:
    wordvec = [r[c] for c in columns[4:]]
    r['Corr'] = np.corrcoef(wordvec,avg_vector[r['Word']])[0,1]

In [46]:
with open('output.csv','w') as f:
    w = csv.DictWriter(f,columns)
    w.writeheader()
    for rr in rows:
        w.writerow(rr)